In [2]:
import os
import csv
import pickle
import pandas as pd
import numpy as np
from kloppy import statsbomb
from tqdm import tqdm
from mplsoccer.pitch import Pitch
import matplotlib.pyplot as plt
import seaborn as sns
from mplsoccer import Sbopen

import warnings
warnings.filterwarnings('ignore')

/Users/jaeminiman/anaconda3/envs/pytorch/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### 리그 데이터 풀 시즌으로 5개 정도?
|idx| comp id | season id | comp name | year | 경기 횟수 | 특징 |
|--|--|--|--|--|--|--|
|1 |2  |27|PL      |15_16  |380 | 380경기 전경기 있음  |
|2 |11 |42| LALIGA | 19_20 | 33 | 바르셀로나 경기만 있음 |
|3 |11 |4 | LALIGA | 18_19 | 34 | 바르셀로나 경기만 있음 |
|4 |11 |1 | LALIGA | 17_18 | 36 | 바르셀로나 경기만 있음 |
|5 |7  |27|Ligue1  |15_16  |377 | 3경기 빼고 있음      |


In [ ]:
print("총 경기 데이터 수 : {} 경기".format(380 + 33 + 34 + 36 + 377))

#### Match id 불러오기

In [ ]:
match_id_json = pd.DataFrame()
for comp_id, season_id in [(2, 27), (11, 42), (11, 4), (11, 1), (7, 27)]:
    t_df = pd.read_json("https://raw.githubusercontent.com/jmlee8939/open-data/master/data/matches/{}/{}.json".format(comp_id, season_id))
    t_df['comp_id'] = comp_id
    t_df['season_id'] = season_id
    match_id_json = pd.concat([match_id_json, t_df], axis=0)
match_id_json.reset_index(inplace=True)

In [ ]:
print("총 데이터 셋 : {} 개".format(len(match_id_json)))

In [ ]:
match_id = match_id_json['match_id'].to_list()

#### mplsoccer 라이브러리 활용해서 전처리

In [ ]:
def filter(event_df) :
    # 경기 상황 변수(Game states)로 활용될 이벤트 종류
    # 슛
    shot_bool = (event_df.type_name == 'Shot')
    # 옐로우 카드
    yellow_bool = (event_df.foul_committed_card_name == 'Yellow Card')
    # 레드 카드
    red_bool = (event_df.foul_committed_card_name == 'Red Card')
    # 골
    goal_bool = (event_df.outcome_name == 'Goal')
    # 파울
    foul_bool = (event_df.type_name == 'Foul Committed')
    # 코너킥
    corner_bool = (event_df.sub_type_name == 'Corner')
    # 오프사이드
    offside_bool = (event_df.outcome_name == 'Pass Offside')

    output_df = event_df[shot_bool | yellow_bool | red_bool | goal_bool | foul_bool | corner_bool | offside_bool].copy()
    output_df['feature_type'] = 0
    output_df.loc[shot_bool,'feature_type'] = 'Shot'
    output_df.loc[foul_bool, 'feature_type'] = 'Foul Committed'
    output_df.loc[yellow_bool, 'feature_type'] = 'Yellow Card'
    output_df.loc[red_bool, 'feature_type'] = 'Red Card'
    output_df.loc[goal_bool, 'feature_type'] = 'Goal'
    output_df.loc[corner_bool, 'feature_type'] = 'Corner'
    output_df.loc[offside_bool, 'feature_type'] = 'Pass Offside'
    return (output_df[['period', 'minute', 'team_name', 'feature_type']])

In [ ]:
parser = Sbopen()
match_event_dict = {}
fail_match_list = []

for i in tqdm(range(len(match_id_json)), "전처리 진행한 경기 수", mininterval = 0.1) :
    t_df = match_id_json.loc[i,['match_date', 'home_score', 'away_score']]
    
    t_match_info = match_id_json.loc[i,:]

    t_season_id = t_match_info['season_id']
    t_comp_id = t_match_info['comp_id']
    t_match_id = t_match_info['match_id']

    t_df['home_team'] = t_match_info['home_team']['home_team_name']
    t_df['away_team'] = t_match_info['away_team']['away_team_name']
    
    try : 
        event_df, _, _, _ = parser.event(t_match_id)
        event_df = filter(event_df)
        
        match_event_dict[t_match_id] = [t_df, event_df]
    except :
        fail_match_list.append([t_comp_id, t_season_id, t_match_id])

In [ ]:
print("정상적으로 전처리된 최종 경기수 : {} 경기".format(len(match_event_dict.keys())))

#### 1차 전처리 한 데이터 pkl 형태로 저장

In [ ]:
#with open('./data/preprocessing_1.pkl', 'wb') as f:
#    pickle.dump(match_event_dict, f)

In [ ]:
mydict = pd.read_pickle('./data/preprocessing_1.pkl')

#### Elo-Rating 전처리

1. 전 시즌 순위로 1200-1800 까지 균등 배분
2. 시즌 중 경기결과는 elo-rating 에 반영 

https://eloratings.net/ 의 공식 

$$ R_n = R_o + K * (W - W_e) $$
$$ W_e = 1 / (10^{(-dr/400)} + 1) $$

In [ ]:
def elo(h_elo, a_elo, result):
    dr = h_elo - a_elo

    if result > 0:
        h_result = 1
        a_result = 0
    elif result == 0:
        h_result = 0.5
        a_result = 0.5
    else :
        h_result = 0
        a_result = 1

    h_we = 1/(10**(-dr/400)+1)
    a_we = 1/(10**(dr/400)+1)

    # K value for league match is 20

    h_e = h_elo + 20*(h_result - h_we)
    a_e = a_elo + 20*(a_result - a_we)

    return int(h_e), int(a_e)

In [ ]:
elo(1700, 1400, 0)

### 필요한 시즌 경기 데이터
- PL 1516
- LALIGA 1718, 1819, 1920
- Ligue1 1516

#### 전 시즌 순위

|전시즌 순위 | PL 1415 | LIGA 1617 | LIGUE1 1415 | elo-rating |
|--|--|--|--|--|
| 1| Chelsea | Real Madrid | Paris Saint-Germain | 1800|
| 2| Manchester City | Barcelona | Lyon | 1766|
| 3| Arsenal | Atletico Madrid | Monaco | 1733|
| 4| Manchester United | Sevilla | Marseille | 1700|
| 5| Tottenham | Villarreal | Saint-Etienne | 1666|
| 6| Liverpool | Real Sociedad | Bordeaux | 1633|
| 7| Southampton | Athletic Bilbao | Montpellier | 1600|
| 8| Swansea | Espanyol | Lille | 1566|
| 9| Stoke | Deportivo Alaves  | Rennes | 1533|
| 10| Crystal Palace | Eibar | Guingamp | 1500|
| 11| Everton | Malaga | Nice | 1466|
| 12| West Ham | Valencia | SC Bastia | 1433|
| 13| West Bromwich Albion | Celta Vigo | Caen | 1400|
| 14| Leicester | Las Palmas | Nantes | 1366|
| 15| Newcastle United | Real Betis | Reims | 1333|
| 16| Sunderland | Deportivo La Coruna | Lorient | 1300|
| 17| Aston Villa | Leganes | Toulouse | 1266|
| 2부| Bournemouth | Getafe | Angers | 1200|
| 2부| Norwich | Athletic Bilbao | GFC Ajaccio | 1200|
| 2부| Watford | Rayo Vallecano | Troyes | 1200|
| 2부| | SD Huesca || 1200|
| 2부| | Real Valladolid || 1200|
| 2부| | Levante || 1200|
| 2부| | Granada || 1200|
| 2부| | Mallorca || 1200|
| 2부| | Osasuna || 1200|

##### 시즌 시작 전 elo-rating initital value

In [ ]:
elo_dict_PL = {'Chelsea' : 1800, 'Manchester City' : 1766, 'Arsenal' : 1733, 'Manchester United' : 1700, 'Tottenham' : 1666, 'Liverpool' : 1633, 'Southampton' : 1600,
            'Swansea' : 1566, 'Stoke' : 1533, 'Crystal Palace' : 1500, 'Everton' : 1466, 'West Ham' : 1433, 'West Bromwich Albion' : 1400,
            'Leicester' : 1366, 'Newcastle United' : 1333, 'Sunderland' : 1300, 'Aston Villa' : 1266, 'Bournemouth' : 1200, 'Norwich' : 1200, 'Watford' : 1200}

elo_dict_LIGA = {'Real Madrid' : 1800, 'Barcelona' : 1766, 'Atletico Madrid' : 1733, 'Sevilla' : 1700, 'Villarreal' : 1666, 'Real Sociedad' : 1633, 'Athletic Bilbao' : 1600,
            'Espanyol' : 1566, 'Deportivo Alaves' : 1533, 'Eibar' : 1500, 'Malaga' : 1466, 'Valencia' : 1433, 'Celta Vigo' : 1400,
            'Las Palmas' : 1366, 'Real Betis' : 1333, 'Deportivo La Coruna' : 1300, 'Leganes' : 1266, 'Getafe' : 1200, 
            'Levante' : 1200, 'Rayo Vallecano' : 1200, 'SD Huesca' : 1200, 'Real Valladolid' : 1200, 'Girona' : 1200, 'Granada' : 1200,
            'Mallorca' : 1200, 'Osasuna' : 1200
}

elo_dict_LIGUE1 = {'Paris Saint-Germain' : 1800, 'Lyon' : 1766, 'Monaco' : 1733, 'Marseille' : 1700, 'Saint-Etienne' : 1666, 'Bordeaux' : 1633, 'Montpellier' : 1600,
            'Lille' : 1566, 'Rennes' : 1533, 'Guingamp' : 1500, 'Nice' : 1466, 'SC Bastia' : 1433, 'Caen' : 1400,
            'Nantes' : 1366, 'Reims' : 1333, 'Lorient' : 1300, 'Toulouse' : 1266, 'Angers' : 1200, 'GFC Ajaccio' : 1200, 'Troyes' : 1200}


#### 경기 데이터 전처리

In [ ]:
def preprocess_match_data(df):
    # 날짜 타입 변경
    month_dict = {'Jan':1, 'Feb':2, 'Mac':3, 'Apr':4, 'Mei':5, 'Jun':6, 'Jul':7, 'Ago':8, 'Sep':9, 'Okt':10, 'Nov':11, 'Des':12}
    t_df = df[['date', 'home', 'away', 'full_home_score', 'full_away_score']]
    t_df['year'] = [i.split('-')[2] for i in t_df['date']]
    t_df['month'] = [month_dict[i.split('-')[1]] for i in t_df['date']]
    t_df['day'] = [i.split(' ')[1].split('-')[0] for i in t_df['date']]
    t_df['date'] = ['/'.join([str(t_df.iloc[i,:]['month']), t_df.iloc[i,:]['day'], t_df.iloc[i,:]['year']])  for i in range(len(t_df))]
    t_df['date'] = pd.to_datetime(t_df['date'])
    t_df['result'] = t_df['full_home_score'] - t_df['full_away_score']
    t_df.sort_values(by='date', inplace=True)
    t_df.reset_index(drop=True, inplace=True)
    teams = list(set(t_df['home']) | set(t_df['away']))
    return t_df, teams

In [ ]:
PL1516 = pd.read_csv("./data/league_data/PL1516_match.csv", index_col=0)
LIGA1718 = pd.read_csv("./data/league_data/LIGA1718_match.csv", index_col=0)
LIGA1819 = pd.read_csv("./data/league_data/LIGA1819_match.csv", index_col=0)
LIGA1920 = pd.read_csv("./data/league_data/LIGA1920_match.csv", index_col=0)
LIGUE1_1516 = pd.read_csv("./data/league_data/LIGUE1_1516_match.csv", index_col=0)

In [ ]:
LIGA = pd.concat([LIGA1718, LIGA1819, LIGA1920], axis=0).reset_index(drop=True)

In [ ]:
PL_df, pl_teams = preprocess_match_data(PL1516)
LIGA_df, liga_teams = preprocess_match_data(LIGA)
LIGUE1_df, ligue1_teams = preprocess_match_data(LIGUE1_1516)

#### 리그 별 elo-rating 변동 값 구하기

In [ ]:
def elo_rating_df(elo_dictionary, match_df):
    elo_dict = elo_dictionary.copy()
    teams = list(elo_dict.keys())
    elo_rating_df = pd.DataFrame(index = pd.date_range(start=match_df['date'][0].to_pydatetime(), end=match_df['date'][len(match_df)-1].to_pydatetime()), 
                                 columns=teams)
    
    # 경기 결과 있는 날 elo 업데이트
    for i in elo_rating_df.index:
        match_day = i.strftime('%Y-%m-%d')
        boolean = match_df['date'] == match_day
        t_df = match_df[boolean].reset_index(drop=True)
        t_elo_dict = elo_dict.copy()
        
        for j in range(boolean.sum()):
            home_team = t_df.loc[j, 'home']
            away_team = t_df.loc[j, 'away']
            result = t_df.loc[j, 'result']
            
            h_e, a_e = elo(elo_dict[home_team], elo_dict[away_team], result)
            t_elo_dict[home_team] = h_e
            t_elo_dict[away_team] = a_e
    # 경기 전 elo 입력
        elo_rating_df.loc[i] = elo_dict
    # 경기 후 elo 업데이트    
        elo_dict = t_elo_dict

    return elo_rating_df

In [ ]:
elo_PL = elo_rating_df(elo_dict_PL, PL_df)
elo_LIGA = elo_rating_df(elo_dict_LIGA, LIGA_df)
elo_LIGUE1 = elo_rating_df(elo_dict_LIGUE1, LIGUE1_df)

#### elo 값 확인

In [ ]:
plt.figure(figsize=(15, 5))
g = sns.lineplot(data=elo_PL)
sns.move_legend(g, loc=(1.01,0))
g.set_title('Premier League')

In [ ]:
plt.figure(figsize=(15, 5))
g = sns.lineplot(data=elo_LIGA)
sns.move_legend(g, loc=(1.01,0))
g.set_title('La Liga')

In [ ]:
plt.figure(figsize=(15, 5))
g = sns.lineplot(data=elo_LIGUE1)
sns.move_legend(g, loc=(1.01,0))
g.set_title('League 1')

In [ ]:
all_elo = pd.concat([elo_PL, elo_LIGA, elo_LIGUE1], axis=1)

In [ ]:
all_elo.head()

In [ ]:
all_elo.fillna(method='ffill', inplace=True)
all_elo.fillna(method='bfill', inplace=True)

#### 경기 데이터에 elo rating 값 추가 

In [ ]:
from difflib import get_close_matches

In [ ]:
all_teams = set([mydict[i][0]['home_team'] for i in mydict.keys()]) | set([mydict[i][0]['away_team'] for i in mydict.keys()])
all_teams_whos = pl_teams + liga_teams + ligue1_teams
name_bridge_dict = {i : get_close_matches(i, all_teams_whos, 1, 0)[0] for i in list(all_teams)}

In [ ]:
for i in mydict.keys():
    t_df = mydict[i][0]
    m_date = t_df['match_date']
    h_team = t_df['home_team']
    a_team = t_df['away_team']
    mydict[i][0]['home elo'] = all_elo.loc[pd.to_datetime(m_date), name_bridge_dict[h_team]]
    mydict[i][0]['away elo'] = all_elo.loc[pd.to_datetime(m_date), name_bridge_dict[a_team]]

In [ ]:
#with open('./data/preprocessing_2.pkl', 'wb') as f:
#    pickle.dump(mydict, f)

In [3]:
mydict = pd.read_pickle('./data/preprocessing_2.pkl')

#### 시간대 별 경기 상황 변수 전처리
- 한 경기를 20 등분 (5분 마다)
- 누적 변수
    - 득점
    - 레드카드
    - 옐로우 카드
- 그 외 변수
    - 10 분 내 슛 횟수
    - 10 분 내 오프사이드 횟 수
    - 10 분 내 파울 수
    - 10 분 내 코너킥 수

In [4]:
t_match_result, t_match_event = mydict[list(mydict.keys())[6]]

In [5]:
t_match_result

match_date              2016-05-15
home_score                       1
away_score                       1
home_team     West Bromwich Albion
away_team                Liverpool
home elo                    1393.0
away elo                    1568.0
Name: 6, dtype: object

In [6]:
def time_seg(minute, period):
    # 전반(10), 후반(10) 하여 1-20 frame 으로 분할
    if period == 1:
        seg = minute // 5
        if seg >= 10 :
            seg = 9
    else : 
        seg = minute // 5
        if seg <= 9 :
            seg = 10
        elif seg >= 20:
            seg = 19
    return seg+1

In [7]:
def game_state_preprocessing(event_df, result_df):
    # home & awayteam
    home_team = result_df['home_team']
    away_team = result_df['away_team']

    # 전체 누적 값, 이전 10분 값 columns 구분하여 전처리
    cumsum_col = ['Goal', 'Yellow Card', 'Red Card']
    rangesum_col = ['Corner', 'Foul Committed', 'Shot', 'Pass Offside']


    t_event_event = event_df.copy()
    t_event_event['frame'] = [time_seg(t_event_event['minute'].iloc[i], t_event_event['period'].iloc[i]) for i in range(len(t_event_event))]
    table = pd.pivot_table(t_event_event, values ='period', index=['team_name', 'frame'],
                        columns=['feature_type'], aggfunc="count").fillna(0)

    # 해당 경기에서 발생하지 않은 이벤트 column 추가
    table[list(set(cumsum_col + rangesum_col).difference(table.columns))] = 0
    table = table[['Yellow Card', 'Red Card', 'Goal', 'Corner', 'Foul Committed', 'Shot', 'Pass Offside']]
    # 슛팅 숫자 = 슛 이벤트 + 골 이벤트
    table['Shot'] = table['Shot'] + table['Goal']

    h_table = table.loc[[j[0] == home_team for j in table.index]].reset_index('frame').set_index('frame')
    a_table = table.loc[[j[0] == away_team for j in table.index]].reset_index('frame').set_index('frame')
    
    zero_df = pd.DataFrame(columns=table.columns, index=np.arange(1,21), data=0)
    h_table = zero_df + h_table
    a_table = zero_df + a_table
    h_table.fillna(0, inplace=True)
    a_table.fillna(0, inplace=True)

    h_table[cumsum_col] = h_table[cumsum_col].cumsum()
    h_table[rangesum_col] = h_table[rangesum_col] + h_table[rangesum_col].shift(1).fillna(0)

    a_table[cumsum_col] = a_table[cumsum_col].cumsum()
    a_table[rangesum_col] = a_table[rangesum_col] + a_table[rangesum_col].shift(1).fillna(0)
    return h_table, a_table

In [8]:
final_dict ={}
for i in tqdm(mydict.keys(), "최종 전처리 진행한 경기 수", mininterval=0.1):
    final_dict[i] = []
    final_dict[i].append(mydict[i][0])
    h_tab, a_tab = game_state_preprocessing(mydict[i][1], mydict[i][0])
    final_dict[i].append(h_tab)
    final_dict[i].append(a_tab)

최종 전처리 진행한 경기 수: 100%|██████████| 814/814 [00:03<00:00, 264.20it/s]


In [9]:
with open('./data/final_data.pkl', 'wb') as f:
    pickle.dump(final_dict, f)